In [1]:
] activate . 

  Activating project at `~/GithubRepos/certified-pose`


In [64]:
# ] st returns:
# Status `~/GithubRepos/certified-pose/Project.toml`
#   [6e4b80f9] BenchmarkTools v1.3.2
#   [8729bc32] GraduatedNonConvexity v0.1.0 `https://github.com/dev10110/GraduatedNonConvexity.jl#main`
#   [91a5bcdd] Plots v1.38.3
#   [6038ab10] Rotations v1.3.4
#   [90137ffa] StaticArrays v1.5.12
#   [37e2e46d] LinearAlgebra

In [65]:
] st

Status `~/GithubRepos/certified-pose/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [8729bc32] GraduatedNonConvexity v0.1.0 `https://github.com/dev10110/GraduatedNonConvexity.jl#main`
  [91a5bcdd] Plots v1.38.3
  [6038ab10] Rotations v1.3.4
  [90137ffa] StaticArrays v1.5.12
  [37e2e46d] LinearAlgebra


In [4]:
using LinearAlgebra, Random, Rotations, GraduatedNonConvexity, StaticArrays, BenchmarkTools

In [5]:
using Plots

In [6]:
gr()

Plots.GRBackend()

In [7]:
## generate points

In [8]:
function rotdist(R1, R2)
    s = (tr(R1 * R2') - 1) / 2 
    s = clamp(s, -1.0,1.0) # to correct for potential float pt errors
    return acos(s)
end

rotdist (generic function with 1 method)

In [9]:
function ransac(N, data, solver, res, c, n; verbose=false, min_inlier_ratio=0.1, max_iterations=1000, initial_guess=solver(ones(N), data))
    
    iter = 0
    bestErr = Inf
    x_best = copy(initial_guess)
    rs = res(x_best, data)
    best_error = mapreduce(r->max(r^2, c^2),  +, rs)
    #@show best_error

    for iter = 1:max_iterations
        
        # sample randomly
        sample = rand(1:N, n)
        sample_w = zeros(N)
        sample_w[sample] .= 1
        # solve for the best solution on these points
        sample_x = solver(sample_w, data)
        
        # check for inliners
        rs = res(sample_x, data)
        inliers = [i for i=1:N if abs(rs[i]) <= c]
        #@show length(inliers)
        if length(inliers) > min_inlier_ratio * N
            #@show length(inliers)
            # fit to all inliers
            inlier_w = zeros(N)
            inlier_w[inliers] .= 1
            inlier_x = solver(inlier_w, data)
            inlier_rs = res(inlier_x, data)
            inlier_error = mapreduce(r-> max(r^2, c^2), +, inlier_rs[inliers])
            #@show inlier_error, best_error
            if inlier_error < best_error
                #@show "replacing"
                x_best = inlier_x
                best_error = inlier_error
            end
        end
    end
    
    return x_best
end
    

ransac (generic function with 1 method)

In [11]:
module PoseEstimation

using LinearAlgebra, StaticArrays, GraduatedNonConvexity, Parameters

# all quaternions are 

SV3{F} = SVector{3,F}
SV4{F} = SVector{4,F}
Quaternion{F} = SVector{4,F}

# returns qa ∘ qb
function quatprod(qa, qb)
    return Ω1(qa) * qb
end

# returns inv(q)
function quatinv(q::Quaternion)
    return Quaternion(-q[1], -q[2], -q[3], q[4])
end

# function vechat(q)
#     if length(q) == 3
#         return q[1], q[2], q[3], zero(q[3])
#     end
#     if length(q) == 4
#         return q[1], q[2], q[3], q[4]
#     end
# end

# function vechat(q::SV3{F}) where {F}
#     return q[1], q[2], q[3], zero(F)
# end

# function vechat(q::SV4{F}) where {F}
#     return q[1], q[2], q[3], q[4]
# end


# returns operator Ω1(q) such that q ∘ v = Ω1(q) v̂
function Ω1(q::Quaternion)# left mul operator

    q1 = q[1]
    q2 = q[2]
    q3 = q[3]
    q4 = q[4]
    
    return @SMatrix [
        [ q4 ;; -q3 ;;  q2 ;; q1] ;
        [ q3 ;;  q4 ;; -q1 ;; q2] ;
        [-q2 ;;  q1 ;;  q4 ;; q3] ;
        [-q1 ;; -q2 ;; -q3 ;; q4]
    ]
    
end
function Ω1(q::SV3)
    return Ω1(Quaternion(q..., 0))
end

# returns operator Ω2(q) such that v ∘ q = Ω2(q) v̂
function Ω2(q::Quaternion) # right mul operator
    
    q1 = q[1]
    q2 = q[2]
    q3 = q[3]
    q4 = q[4]
    
    return @SMatrix [
        [ q4 ;;  q3 ;; -q2 ;; q1] ;
        [-q3 ;;  q4 ;;  q1 ;; q2 ] ;
        [ q2 ;; -q1 ;;  q4 ;; q3 ] ;
        [-q1 ;; -q2 ;; -q3 ;; q4]
    ]
    
end

function Ω2(q::SV3)
    return Ω2(Quaternion(q..., 0))
end

function quat_to_rot(q::Quaternion{F}) where {F}
    
    R̂ = Ω1(q) * Ω2(quatinv(q))
    
    R = R̂[1:3, 1:3]
    
    return SMatrix{3,3,F,9}(R)
end

function construct_Q(N, a::AF, b::AF, w::VF) where {F, AF<:AbstractArray{F}, VF<:AbstractVector{F}}
    # construct Q matrix
    Q = zero(MMatrix{4,4, F, 16})

    @inbounds for i=1:N
        if w[i] != 0
            qa = Quaternion(a[1, i], a[2, i], a[3, i], zero(F))
            qb = Quaternion(b[1, i], b[2, i], b[3, i], zero(F))
            Ω1b = Ω1(qb)
            Ω2a = Ω2(qa)
            Qi = Ω1b' * Ω2a
            wQiQi = w[i] * Qi + Qi'
            Q .-= wQiQi
        end
    end
    return Q
end

"""
    estimate_R(a, b, w)

solves the problem

R^* = min sum_{i=1}^N w_i ||b_i - R a_i||^2

in closed form
"""
function _estimate_R(a::AF, b::AF, w=ones(F, size(a, 2))) where {F, AF <: AbstractArray{F}}
    """
    method: cast the problem in quaternions
    write the problem as min q^T Q q such that ||q|| = 1
    notice the problem is an eigenvector problem
    return eigenvector with smallest eigenvalue
    """
    
    @assert size(a) == size(b)
    N = size(a, 2)
    @assert length(w) == N
    
    Q = construct_Q(N, a, b, w)
    
    # get min eigenvector 
    q = eigvecs(Q)[:,1] |> real |> Quaternion{F} 

    # convert to rotation matrix
    R = quat_to_rot(q)
    
    return R
    
end

"""
    estimate_t(s, w)

solves the problem

R^* = min sum_{i=1}^N w_i ||s_i - t||^2

in closed form

"""
function _estimate_t(s, w=ones(size(s,2)))
    
    D, N = size(s) # t \in R^D, and there are N points to match
    
    P = sum(w)*I(D)
    q = sum(w[i] * s[:, i] for i in 1:N)
    
    return P \ q
end

abstract type PairingMethod end

@with_kw struct Star <:PairingMethod
    n::Integer = 1
end

@with_kw struct Complete <:PairingMethod
    frac::Float64 = 1.0
end

function make_pairs(m::Star, N)
    is = ones(Int, N-1) * m.n
    js = [j for j=1:N if j != m.n]
    return is, js
end

function make_pairs(m::Complete, N::T) where {T}
    
    is = T[]
    js = T[]
    for i=1:N, j=1:N
        if i > j && (m.frac == 1 || rand() < m.frac)
            push!(is, i)
            push!(js, j)
        end
    end
    
    return is, js
end

abstract type LsqMethod end

struct LS <: LsqMethod
end

@with_kw struct GM <: LsqMethod
    c̄
    max_iterations = 1000
    μ_factor = 1.4
    verbose=false
    rtol=1e-6
end
GM(c, kwargs...) = GM(c̄=c; kwargs...)

@with_kw struct TLS <: LsqMethod
    c̄
    max_iterations = 1000
    μ_factor = 1.4
    rtol=1e-6
    verbose = false
end
TLS(c) = TLS(c̄=c)


function wls_solver_R(w, data)
    a, b = data
    return _estimate_R(a, b, w)
end

function residuals_R(R, data)
    a, b = data
    N = size(a, 2)
    δ = b - R*a
    return [norm(δ[:, i]) for i=1:N]
end

function wls_solver_t(w, s)
    return _estimate_t(s, w)
end

function residuals_t(t, s)
    return [norm(s[:, i] - t) for i=1:size(s, 2)]
end


function estimate_R(method::LS, a, b)
    R = _estimate_R(a, b)
    return R
end

function estimate_R(method::TLS, a, b)
    N = size(a, 2)
    data = (a, b)
    R = GNC_TLS(N, data, wls_solver_R, residuals_R, method.c̄; 
        max_iterations = method.max_iterations,
        μ_factor = method.μ_factor,
        verbose=method.verbose,
        rtol = method.rtol
    )
    return R
end

function estimate_R(method::GM, a, b)
    N = size(a, 2)
    data = (a, b)
    R = GNC_GM(N, data, wls_solver_R, residuals_R, method.c̄; 
        max_iterations = method.max_iterations,
        μ_factor = method.μ_factor,
        verbose=method.verbose,
        rtol = method.rtol
    )
    return R
end

function estimate_t(method::LS, p1,p2,R)
    s = p2 - R*p1
    t = _estimate_t(s)
    return t
end

function estimate_t(method::TLS, p1,p2,R)

    s = p2 - R * p1
    N = size(s, 2)
    
    R = GNC_TLS(N, s, wls_solver_t, residuals_t, method.c̄; 
        max_iterations = method.max_iterations,
        μ_factor = method.μ_factor,
        verbose=method.verbose,
        rtol = method.rtol
    )
    return R
end

function estimate_t(method::GM, p1, p2, R)
    s = p2 - R * p1
    N = size(s, 2)
    
    R = GNC_TLS(N, s, wls_solver_t, residuals_t, method.c̄; 
        max_iterations = method.max_iterations,
        μ_factor = method.μ_factor,
        verbose=method.verbose,
        rtol = method.rtol
    )
    return R
end

function estimate_Rt(p1, p2; method_pairing::PairingMethod, method_R::LsqMethod, method_t::LsqMethod)
    
    N = size(p1, 2)
    
    is, js = make_pairs(method_pairing, N)
    
    a = p1[:, is] - p1[:, js]
    b = p2[:, is] - p2[:, js]
    
    R = estimate_R(method_R, a, b)
    
    t = estimate_t(method_t, p1, p2, R)
    
    return R, t
    
end


σmin(A) = sqrt(max(0, eigmin(A'*A)))


# note, this function scales as N^4
# only pass in inliers to p1, i.e. pass in p1[:, inliner_idx]
# this function implements different maths than in Teaser
# since the proof in Teaser is wrong
# a (probably) good anytime approximation will be to run this function with random sets of 4 inds
# and take the lowest bound produced. This approximation will still be an upper-bound to the error
function ϵR(p1, β)
    
    N = size(p1, 2)
    
    best_bound = Inf

    @views @inbounds for i=1:N
        
        PI = SMatrix{3,3}(p1[:, [i,i,i]])
        
        for j=(i+1):N, h=(j+1):N, k=(h+1):N
        
           U = SMatrix{3,3}(p1[:, [j,h,k]]) - PI

           σ = σmin(U)
            
           if σ > 0
                bound = 2*sqrt(3)*(2β) / σ
                best_bound = min(bound, best_bound)
           end
            
        end
    end
    
    return best_bound
    
end

function ϵt(β)
    return (9 + 3*sqrt(3))*β
end


end

Main.PoseEstimation

In [12]:
using .PoseEstimation
PE = PoseEstimation

Main.PoseEstimation

In [13]:
Random.seed!(42);

In [14]:

## construct ground truth data


N = 1_000 # number of correspondences
p1 = randn(3, N)

# generate a ground-truth pose
R_groundtruth = rand(RotMatrix{3})

# generate a ground-truth translation
t_groundtruth = randn(3)

# generate true p2
p2 = R_groundtruth * p1  .+ t_groundtruth

3×1000 Matrix{Float64}:
 -1.00473   -0.282145  -3.1916    …  -1.40873    -0.590944  -2.07617
 -1.14969   -0.657871  -2.11146       0.38221    -2.50974   -0.960117
 -0.128386  -0.557652   0.730906      0.0422261  -0.593872   0.063647

In [15]:
# make noisy measurements
β = 0.01
p2_noisy = copy(p2)
for i=1:N
    p2_noisy[:, i] += β*(2*rand(3).-1)
end

# add outliers to some% of data
inds = [i for i=2:N if rand() < 0.70]
for i=inds
    p2_noisy[:, i] += 3*randn(3)
end

In [16]:
# set maximum residual of inliers
# this number should be computed based off β
c̄ = 0.005

0.005

In [37]:
# normal least squares is quite off

In [38]:
@time R_ls, t_ls = PE.estimate_Rt(p1, p2_noisy;
    method_pairing=PE.Star(),
    method_R=PE.LS(),
    method_t=PE.LS())

@show rotdist(R_ls, R_groundtruth) * 180 / π
@show norm(t_ls - t_groundtruth)

  0.000413 seconds (3.56 k allocations: 518.359 KiB)
(rotdist(R_ls, R_groundtruth) * 180) / π = 6.559942365519717
norm(t_ls - t_groundtruth) = 0.18690212342874568


0.18690212342874568

In [39]:
# TLS works quite well

In [40]:
@time R_tls, t_tls = PE.estimate_Rt(p1, p2_noisy;
    method_pairing=PE.Star(),
    method_R=PE.TLS(c̄ = 0.07), # TODO: fix c̄, put in the theoretically correct value based on β
    method_t=PE.TLS(c̄ = 0.07)
)

@show rotdist(R_tls, R_groundtruth) * 180 / π
@show norm(t_tls - t_groundtruth)

  0.015191 seconds (171.26 k allocations: 16.057 MiB)
(rotdist(R_tls, R_groundtruth) * 180) / π = 0.1373080874015569
norm(t_tls - t_groundtruth) = 0.0005290940925725104


0.0005290940925725104

In [41]:
# even when we overestimate the magnitude of noise
@time R_tls, t_tls = PE.estimate_Rt(p1, p2_noisy;
    method_pairing=PE.Star(),
    method_R=PE.TLS(c̄ = 0.25),
    method_t=PE.TLS(c̄ = 0.25))

@show rotdist(R_tls, R_groundtruth) * 180 / π
@show norm(t_tls - t_groundtruth)

  0.012534 seconds (139.10 k allocations: 12.971 MiB)
(rotdist(R_tls, R_groundtruth) * 180) / π = 0.1373080874015569
norm(t_tls - t_groundtruth) = 0.0005290940925725104


0.0005290940925725104

In [42]:
# GM doesnt work as well - probably just not as well tuned

In [43]:
@time R_gm, t_gm = PE.estimate_Rt(p1, p2_noisy;
    method_pairing=PE.Star(),
    method_R=PE.GM(0.07),
    method_t=PE.GM(0.07))

@show rotdist(R_gm, R_groundtruth) * 180 / π
@show norm(t_gm - t_groundtruth)

  0.050644 seconds (276.43 k allocations: 24.658 MiB, 56.84% gc time)
(rotdist(R_gm, R_groundtruth) * 180) / π = 6.506343919389801
norm(t_gm - t_groundtruth) = 0.025151107561736087


0.025151107561736087

In [44]:
# using a complete graph (instead of a star graph) helps, but is (much) slower

In [45]:
@time R_tls, t_tls = PE.estimate_Rt(p1, p2_noisy;
    method_pairing=PE.Complete(),
    method_R=PE.TLS(0.07),
    method_t=PE.TLS(0.07))

@show rotdist(R_tls, R_groundtruth)
@show norm(t_tls - t_groundtruth)

  3.238169 seconds (19.62 M allocations: 2.560 GiB, 14.27% gc time)
rotdist(R_tls, R_groundtruth) = 0.00033243496504911814
norm(t_tls - t_groundtruth) = 0.0004081488784256854


0.0004081488784256854

In [46]:
inliers = [i for i=1:N if !(i in inds)];

In [47]:
## now for the certification

In [48]:
# estimate the error bound for rotation matrices
# this is a bound on the maximum ||R - R̂||_F

In [62]:
@time begin bb=Inf
for i=1:1000
    bb = min(bb, PE.ϵR(p1[:, rand(inliers, 4)], β))
end
bb
end

  0.001226 seconds (10.00 k allocations: 687.500 KiB)


0.03343246392961011

In [50]:
PE.ϵt(β)

0.14196152422706632

In [51]:
# woah thats huge!!
# a 1 cm measurement error on each point correspinds to 14 cm error ball!
# the derivation in the paper is also suspect - I think we can improve it

In [52]:
# compare it to the actual error

In [53]:
norm(R_ls - R_groundtruth)

0.16182855493653758

In [54]:
norm(R_tls - R_groundtruth)

0.00047013403319433355

In [55]:
norm(R_tls - R_groundtruth) < bb

true

In [56]:
norm(R_gm - R_groundtruth)

0.16050775098694672